In [1]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from urllib.request import urlretrieve
from scipy.stats import pearsonr

from config.config import DATA_PATH
pd.set_option('display.max_columns', None)

In [2]:
X_train = pd.read_csv(DATA_PATH + '/modeling/X_train.csv')
X_test = pd.read_csv(DATA_PATH + '/modeling/X_test.csv')
y_train = pd.read_csv(DATA_PATH + '/modeling/y_train.csv')
y_test = pd.read_csv(DATA_PATH + '/modeling/y_test.csv')

In [3]:
y_train = y_train['cancelled']
y_test = y_test['cancelled']

In [4]:
X_train.head()

,quarter,month,day_of_month,day_of_week,hour_of_day,scheduled_elapsed_time,distance,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D,temperature_2m_origin,precipitation_origin,snowfall_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snowfall_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_region_encoded,origin_division_encoded,dest_region_encoded,dest_division_encoded,diverted,code_share_flight,is_holiday,dep_window_early afternoon,dep_window_early morning,dep_window_evening,dep_window_late night,dep_window_midday,dep_window_morning,dep_window_night,dep_window_overnight
0,-1.361845,-1.620543,-1.452345,-0.984862,0.189847,-0.031459,1.937362,0.617993,2.272436,-0.369399,1.035771,1.263474,-0.257625,-0.320950,-0.02657,-1.252609,0.753796,2.917779,-0.651917,1.255640,1.595429,-0.571728,-0.455432,-0.045443,-1.369219,0.844045,3.412272,-0.866083,1.420894,1.855956,-0.807241,-0.614674,-0.075479,-1.260454,0.942484,-0.059526,0.0,-0.098763,-0.279899,-0.691314,1.486775,-0.662031,0.439144,-0.384728,-0.061437,0.0,-0.097708,-0.151443,-0.923607,0.334950,-1.025207,0.658845,0.725836,-0.927173,0.758972,0.686502,-0.303425,-0.373972,-0.794790,-1.033742,0,0,0,1,0,0,0,0,0,0,0
1,-0.462343,-0.450140,1.515007,-1.485173,-0.797123,-0.935554,-1.309607,-0.714057,-0.334788,-0.688254,-0.722140,-0.571252,-0.653726,-0.320950,-0.02657,0.282212,-0.720075,-0.326840,-0.544094,-0.748007,-0.447513,-0.567212,-0.455432,-0.045443,0.509924,-0.721897,-0.240825,-0.703666,-0.789019,-0.509619,-0.678066,-0.247148,-0.075479,0.714027,0.140555,-0.059526,0.0,-0.098763,-0.156940,-0.587775,0.382257,1.248121,-0.070664,0.032863,-0.061437,0.0,-0.097708,-0.151443,-0.965718,-0.207061,0.605140,-0.767828,-0.917549,-0.948258,-0.513727,-0.710339,-0.303425,-0.373972,-0.294948,-0.389142,0,0,1,0,0,0,0,0,1,0,0
2,0.437160,0.720262,0.031331,1.016382,0.979423,-0.067265,-0.193030,0.215444,0.056296,0.718457,-0.047557,-0.031627,0.647750,-0.320950,-0.02657,0.172582,0.228256,0.078737,0.592533,0.076750,-0.118006,0.629512,-0.455432,-0.045443,0.076276,-0.160012,-0.147156,0.071196,-0.099885,0.100852,0.096985,-0.614674,-0.075479,0.241605,0.241497,-0.059526,0.0,-0.098763,-0.156940,-0.804266,-0.080100,0.878091,-0.509665,-0.119502,-0.061437,0.0,-0.097708,-0.336215,-0.521212,-0.542592,1.118397,-0.598323,-0.904235,-1.189224,-1.777531,-1.078380,-0.780530,-1.020498,-0.794790,-1.033742,0,0,0,0,0,1,0,0,0,0,0
3,1.336662,1.012863,-1.452345,-0.984862,-0.402335,-0.774428,-1.139360,-0.896203,-0.660691,-0.688254,-0.678728,-0.598233,-0.590853,-0.320950,-0.02657,-0.485198,-1.015863,-0.651302,-0.822635,-0.863602,-0.645217,-0.797525,-0.455432,-0.045443,-0.393510,-0.753911,-0.521832,-0.764572,-0.755967,-0.585927,-0.824237,-0.614674,-0.075479,-0.206590,0.348047,-0.059526,0.0,-0.098763,-0.156940,-0.423054,-0.388337,-1.282081,-1.019473,0.591532,-0.061437,0.0,-0.097708,-0.336215,-0.563323,0.231709,0.746034,-0.598323,-0.548545,-0.927173,-0.640548,-0.291071,-0.780530,-1.020498,-0.794790,-1.033742,0,1,0,0,0,0,0,1,0,0,0
4,-1.361845,-1.327942,1.172621,1.516693,0.979423,-0.872894,-1.351018,0.966584,0.871053,0.218293,1.557903,1.884043,0.301947,1.237885,-0.02657,-0.485198,-0.143768,-0.326840,-0.229612,0.435048,0.738711,-0.169810,1.070058,-0.045443,-0.429648,0.897922,-0.287659,0.020441,1.482561,0.787632,0.0

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [7]:
logreg_model = LogisticRegression()

logreg_model.fit(X_train, y_train)

y_pred = logreg_model.predict(X_test)

print(classification_report(y_test, y_pred))
print()
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    441319
           1       0.24      0.04      0.07      6416

    accuracy                           0.98    447735
   macro avg       0.61      0.52      0.53    447735
weighted avg       0.98      0.98      0.98    447735


[[440552    767]
 [  6171    245]]


In [9]:
rf_model = RandomForestClassifier()

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

print(classification_report(y_test, y_pred))
print()
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    441319
           1       0.82      0.13      0.23      6416

    accuracy                           0.99    447735
   macro avg       0.90      0.57      0.61    447735
weighted avg       0.99      0.99      0.98    447735


[[441131    188]
 [  5557    859]]


In [10]:
model_important_features = pd.DataFrame(rf_model.feature_importances_,
                                     columns=['feature_importance'],
                                     index=X_test.columns)
model_important_features = model_important_features.sort_values('feature_importance', ascending=False)
model_important_features.head(20)

,feature_importance
cancelled_sum_10D,0.076461
visibility_origin,0.030097
visibility_dest,0.029573
day_of_month,0.029480
temperature_2m_origin,0.028705
temperature_2m_dest,0.027649
n_flights_10D,0.027224
wind_gusts_10m_origin,0.026558
wind_gusts_10m_dest,0.025770
wind_speed_10m_origin,0.025038


In [11]:
gb_model = GradientBoostingClassifier()

gb_model.fit(X_train, y_train)

y_pred = gb_model.predict(X_test)

print(classification_report(y_test, y_pred))
print()
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    441319
           1       0.71      0.09      0.16      6416

    accuracy                           0.99    447735
   macro avg       0.85      0.55      0.58    447735
weighted avg       0.98      0.99      0.98    447735


[[441070    249]
 [  5819    597]]


In [13]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, average_precision_score
from sklearn.model_selection import KFold, GridSearchCV
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

In [16]:
model_results = []

def evaluate_model(model, X_train, y_train, X_test, y_test):
    # Fit the model
    model_fit = model.fit(X_train, y_train)
    
    # Predict
    y_pred = model_fit.predict(X_test)
    y_pred_proba = model_fit.predict_proba(X_test)[:, 1]

    # calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    auprc = average_precision_score(y_test, y_pred_proba)
    
    # Return the metrics
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'auprc': auprc
    }

In [17]:
def grid_search_model(model, params, X_train, y_train):

    kf = KFold(n_splits=3, shuffle=True, random_state=40)

    gs = GridSearchCV(model, param_grid=params, cv=kf, scoring='f1', n_jobs=-1)

    gs_fit = gs.fit(X_train, y_train)

    best_estimator, best_params = gs_fit.best_estimator_, gs_fit.best_params_

    return best_estimator, best_params

In [57]:
from sklearn.model_selection import RandomizedSearchCV

def random_grid_search_model(model, params, X_train, y_train):

    kf = KFold(n_splits=3, shuffle=True, random_state=40)

    rs  = RandomizedSearchCV(model, params, n_iter=50, n_jobs=-1, cv=kf)

    rs_fit = rs.fit(X_train, y_train)

    best_estimator, best_params = rs_fit.best_estimator_, rs_fit.best_params_

    return best_estimator, best_params

In [ ]:
log_reg_param_grid = {
   # 'penalty': ['l1', 'l2'],
    'C': np.logspace(-2, 2, 10),
    'solver': ['saga', 'newton-cholesky'], 
    'max_iter': [700],
}

best_model, best_params = grid_search_model(model=LogisticRegression(), 
                                            params=log_reg_param_grid, 
                                            X_train=X_train, 
                                            y_train=y_train)

logreg_metrics = evaluate_model(best_model, X_train, y_train, X_test, y_test)
logreg_metrics['model'] = 'LogisticRegression_GS'
logreg_metrics['best_params'] = best_params
model_results.append(logreg_metrics)

In [19]:
results_df = pd.DataFrame(model_results)
results_df.sort_values('recall', ascending=False)
results_df

,accuracy,precision,recall,f1,roc_auc,auprc,model,best_params
0,0.984486,0.237103,0.037251,0.064386,0.906372,0.1658,LogisticRegression_GS,"{'max_iter': 700, 'solver': 'newton-cholesky'}"


In [ ]:
rf_param_grid = {
    'n_estimators':[100, 200, 350],
    'criterion': ['gini', 'entropy'],
    'max_depth':[11, 20, None],
    'max_features':['sqrt', 'log2', None],
    'class_weight': [None, 'balanced', 'balanced_subsample']}

best_model, best_params = random_grid_search_model(RandomForestClassifier(), params=rf_param_grid, X_train=X_train, y_train=y_train)
rf_metrics = evaluate_model(best_model, X_train, y_train, X_test, y_test)
rf_metrics['model'] = 'RandomForest_GS'
rf_metrics['best_params'] = best_params
model_results.append(rf_metrics)

In [ ]:
gb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [1, 5, 10],
    'subsample': [0.8, 1.0],
    'max_features': ['sqrt', 'log2', None]}

best_model, best_params = random_grid_search_model(GradientBoostingClassifier(), gb_param_grid, X_train, y_train)
gs_metrics = evaluate_model(best_model, X_train, y_train, X_test, y_test)
gs_metrics['model'] = 'GradientBoosting_GS'
gs_metrics['best_params'] = best_params
model_results.append(gs_metrics)

In [21]:
models = {
    'RandomForest': RandomForestClassifier(random_state=40),
    'GradientBoosting': GradientBoostingClassifier(random_state=40),
}

for model_name, model in models.items():
    metrics = evaluate_model(model, X_train, y_train, X_test, y_test)
    metrics['model'] = model_name
    model_results.append(metrics)
    
undersampler = RandomUnderSampler(random_state=40)
X_train_res, y_train_res = undersampler.fit_resample(X_train, y_train)

models = {
    'RandomForest_undersampler': RandomForestClassifier(random_state=40),
    'GradientBoosting_undersampler': GradientBoostingClassifier(random_state=40),
}

for model_name, model in models.items():
    metrics = evaluate_model(model, X_train_res, y_train_res, X_test, y_test)
    metrics['model'] = model_name
    model_results.append(metrics)

In [ ]:
sm = SMOTE(random_state=40)
X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)

models = {
    'RandomForest_smote': RandomForestClassifier(random_state=40),
    'GradientBoosting_smote': GradientBoostingClassifier(random_state=40),
}

for model_name, model in models.items():
    metrics = evaluate_model(model, X_train_smote, y_train_smote, X_test, y_test)
    metrics['model'] = model_name
    model_results.append(metrics)



In [ ]:
results_df = pd.DataFrame(model_results)
results_df.sort_values('recall', ascending=False)